Google Colab Link (TPU tutorial):
https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/fashion_mnist.ipynb#scrollTo=SaYPv_aKId2d

## Content:
We train our own network on a TPU. The network consists out of stacked conv, pool and dropout layers

## Open To Do's:
* Add a mathematical part, that checks whether or not the stride size of the pooling layer is too large in combination with the amount of layers and will cause an error
* Larger Hyperparameter testing
* Replace dropout by Batch Normalization

In [3]:
#!pip install keras-metrics
import tensorflow as tf
import numpy as np
import time
import os
from datetime import datetime

from io import BytesIO 
from tensorflow.python.lib.io import file_io

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [4]:
def precision(y_true, y_pred):	
    """Precision metric.	
    Only computes a batch-wise average of precision. Computes the precision, a
    metric for multi-label classification of how many selected items are
    relevant.
    """	
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))	
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))	
    precision = true_positives / (predicted_positives + K.epsilon())	
    return precision

def recall(y_true, y_pred):	
    """Recall metric.	
    Only computes a batch-wise average of recall. Computes the recall, a metric
    for multi-label classification of how many relevant items are selected.	
    """	
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))	
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))	
    recall = true_positives / (possible_positives + K.epsilon())	
    return recall

def f1_score(y_true, y_pred):
    """Computes the F1 Score
    Only computes a batch-wise average of recall. Computes the recall, a metric
    for multi-label classification of how many relevant items are selected.	
    """
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return (2 * p * r) / (p + r + K.epsilon())

In [5]:
class cnn_model:
    def __init__(self, x_train, y_train,num_classes, config, name=None):
        """
        initializes the model and defines the graph. There will always be one more
        dense layer than defined.
        """
        # hard features
        self.optimizer = "adam"
        self.loss = "binary_crossentropy"
        
        # mutable features
        self.conv_layers = int(config["conv_layers"])
        self.conv_filters = int(config["conv_filters"])
        self.conv_stride = (int(config["conv_stride"]), int(config["conv_stride"]))
        self.kernel_size = (int(config["kernel_size"]), int(config["kernel_size"]))
        self.pool_size = (int(config["pool_size"]), int(config["pool_size"]))
        self.pool_stride = (int(config["pool_stride"]),int(config["pool_stride"]))
        self.dense_layers = int(config["dense_layers"])
        self.dense_neurons = int(config["dense_neurons"])
        self.dropout_rate_dense = config["dropout_rate_dense"]
        self.learning_rate = config["learning_rate"]
        self.activation_fn = config["activation_fn"]
        # name describes characteristics 
        if bool(name):
            self.name = name
        else:
            self.name = "conv_size_{}_filters_{}_kernel_{}_pool_{}_dense_{}_dropout_\
                dense_{}_lr_{}_act_{}".format(self.conv_layers, self.conv_filters, self.kernel_size[0], 
                self.pool_size[0], self.dense_layers, self.dropout_rate_dense, 
                self.learning_rate, self.activation_fn)
        
        # check whether input is numpy format or a link to google cloud storage
        if isinstance(x_train, str):
            if "gs" in x_train:
                f = BytesIO(file_io.read_file_to_string(x_train, binary_mode=True))
                self.x_train = np.load(f)
        else:
            self.x_train = x_train
            
        if isinstance(y_train, str):
            if "gs" in y_train:
                f = BytesIO(file_io.read_file_to_string(y_train, binary_mode=True))
                self.y_train = np.load(f)
        else:
            self.y_train = y_train
            
        # create train and validation sets
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(self.x_train,
                                                            self.y_train,
                                                            train_size=0.8,
                                                            random_state = 1)
        input_shape = self.x_train.shape[1:]

        # defining the model
        model = Sequential()
        model.add(Conv2D(filters=self.conv_filters,
                         kernel_size=self.kernel_size,
                         activation=self.activation_fn,
                         input_shape=input_shape,
                         padding="SAME",
                         strides=self.conv_stride
                        ))
        model.add(MaxPooling2D(pool_size=self.pool_size,
                 strides=self.pool_stride))
        model.add(BatchNormalization())

        for i in range(self.conv_layers-1):
            model.add(Conv2D(filters=self.conv_filters,
                             kernel_size=self.kernel_size,
                             activation=self.activation_fn,
                             padding="SAME",
                             strides=self.conv_stride
                             #input_shape=input_shape
                          ))
            model.add(MaxPooling2D(pool_size=self.pool_size,
                     strides=self.pool_stride))
            model.add(BatchNormalization())


        model.add(Flatten())
        for i in range(self.dense_layers):
            model.add(Dense(self.dense_neurons, activation=self.activation_fn))
            model.add(Dropout(self.dropout_rate_dense))

        model.add(Dense(num_classes, activation='softmax')) # softmax remains unchanged
        self.model = model   
        
    def train(self, epochs, batch_size, learning_rate=None, optimizer=None, loss=None, verbose=False, on_tpu=False):
        """
        trains the model.

        If the initial config file contained parameters for training then
        these dont have to be defined but can still be overridden
        """ 
        if learning_rate is None:
            learning_rate = self.learning_rate 
        if optimizer is None:
            optimizer = self.optimizer 
        if loss is None:
            loss = self.loss 
                  
        date_time = datetime.now().strftime('%Y-%m-%d-%H%M%S')
        log_name = "gs://data-imr-unisg/logs/{}_{}".format(self.name, date_time)
        tensorboard_callback = TensorBoard(log_dir=log_name,
                                write_graph=True,
                                write_images=True)
        early_stopping_callback = EarlyStopping(monitor="val_loss",
                                                patience=5)
        
        # model has to be compiled differently when on tpu
        if on_tpu:
            
            tpu_model = tf.contrib.tpu.keras_to_tpu_model(self.model,
                                                          strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver("dominique-c-a-paul")))
            tpu_model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['sparse_categorical_accuracy', f1_score])
        
            def train_gen(batch_size):
                """
                generator function for training the model on a tpu
                """
                while True:
                    offset = np.random.randint(0, self.x_train.shape[0] - batch_size)
                    # print(self.x_train[offset:offset+batch_size].shape, self.y_train[offset:offset + batch_size].shape)
                    yield self.x_train[offset:offset+batch_size], self.y_train[offset:offset + batch_size]

            # has to be optimised to really train a epoch with full data
            self.hist = tpu_model.fit_generator(
                train_gen(batch_size),
                epochs=epochs,
                steps_per_epoch=10,
                validation_data=(self.x_val, self.y_val),
                callbacks=[tensorboard_callback]
                )

        else:
            self.y_train = tf.keras.utils.to_categorical(self.y_train, 2 )
            self.y_val = tf.keras.utils.to_categorical(self.y_val, 2 )
            self.model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy', f1])
            self.model.fit(self.x_train, self.y_train, epochs=epochs, batch_size=batch_size, 
                           verbose=verbose, callbacks=[tensorboard_callback], validation_data=(self.x_val, self.y_val))

        def predict(self, x_data):
            cpu_model = tpu_model.sync_to_cpu()
            predictions = cpu_model.predict(x_data)
            return predictions

        
        def save_model(self, file_path):
            raise notImplementedError()
            
        
        
        

## Regular Training

In [16]:
config_v1 = {    
    "conv_layers": 4,
    "conv_filters": 128,
    "conv_stride": 1,
    "kernel_size": 3,
    "pool_size":2,
    "pool_stride": 2,
    "dense_layers": 5,
    "dense_neurons": 20,
    "dropout_rate_dense": 0.2,
    "learning_rate": 1e-04,
    "activation_fn": "relu"
}

x_train_url = 'gs://data-imr-unisg/np_array_files/x_train.npy'
y_train_url = 'gs://data-imr-unisg/np_array_files/class_labels_trainp.npy'

m1 = cnn_model(x_train_url, y_train_url, 2, config_v1)
m1.model.summary()
m1.train(on_tpu=True, epochs=1, batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 299, 299, 128)     3584      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 149, 149, 128)     0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 149, 149, 128)     512       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 149, 149, 128)     147584    
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 74, 74, 128)       0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 74, 74, 128)       512       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 74, 74, 128)       147584    
__________

KeyboardInterrupt: 

In [ ]:
m_opt=cnn_model(x_train_url, y_train_url, 2, config_v2)
hist = m_opt.train(on_tpu=True, epochs=2, batch_size=128)
val_loss = np.max(m_opt.hist.history["val_loss"])
val_accuracy = np.max(m_opt.hist.history["val_loss"])


In [ ]:
m_opt.hist.history

In [ ]:
# !gcloud alpha compute tpus list
# !gcloud compute tpus stop dominique-c-a-paul && gcloud compute tpus start dominique-c-a-paul

## Hyperparameter optimization 

source: 
https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a
https://github.com/WillKoehrsen/hyperparameter-optimization/blob/master/Bayesian%20Hyperparameter%20Optimization%20of%20Gradient%20Boosting%20Machine.ipynb

In [17]:
import csv
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.stochastic import sample
from timeit import default_timer as timer

MAX_EVALS = 2


# File to save first results
out_file = 'bayes_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)
# Write the headers to the file
writer.writerow(['params','run_time', 'val_loss', 'val_accuracy', 'train_loss', 'train_accuracy', 'train_f1'])
of_connection.close()

# hyperparameter optimization with hyperopt
def objective(params):
    m_opt=cnn_model(x_train_url, y_train_url, 2, params)
    print(m_opt.model.summary())
    start = timer()
    m_opt.train(on_tpu=True, epochs=2, batch_size=256)
    run_time = timer() - start
    val_loss = m_opt.hist.history["val_loss"][-1]
    val_accuracy = m_opt.hist.history["val_sparse_categorical_accuracy"][-1]
    val_f1 = m_opt.hist.history["val_f1_score"][-1]
    train_loss = m_opt.hist.history["loss"][-1]
    train_accuracy = m_opt.hist.history["sparse_categorical_accuracy"][-1]
    train_f1 = m_opt.hist.history["f1_score"][-1]
    
    # adding lines to csv
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([params, run_time, val_loss, val_accuracy, train_loss, train_accuracy, train_f1])
    of_connection.close()
    
    return {"loss": val_loss,
            "params": params, 
            "status": hyperopt.STATUS_OK}

# Define the search space
space = {
    "conv_layers": hp.quniform("conv_layers", 4, 8, 1),
    "conv_filters": hp.quniform("conv_filters", 2, 128, 1),
    "conv_stride": hp.quniform("conv_stride", 1, 1, 1),
    "kernel_size": hp.quniform("kernel_size",1, 2, 1),
    "pool_size": hp.quniform("pool_size",1, 3, 1),
    "pool_stride": hp.quniform("pool_stride", 2, 2, 1),
    "dense_layers": hp.quniform("dense_layers", 1, 5, 1),
    "dense_neurons": hp.quniform("dense_neurons", 1, 100, 1),
    "dropout_rate_dense": hp.uniform("dropout_rate_dense",0,1),
    "learning_rate": hp.loguniform('learning_rate', np.log(1e-02), np.log(1e-06)),
    "activation_fn": hp.choice('activation_fn', ["relu"])
}

bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 299, 299, 109)     1417      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 150, 150, 109)     0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 150, 150, 109)     436       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 150, 150, 109)     47633     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 75, 75, 109)       0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 75, 75, 109)       436       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 75, 75, 109)       47633     
__________

                                                   
 9/10 [==========================>...]             
 - ETA: 13s - loss: 1.1611 - sparse_categorical_accuracy: 0.6615 - f1_score: 0.8391
  0%|          | 0/2 [02:28<?, ?it/s, best loss: ?]INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(30,), dtype=tf.int32, name='core_id_70'), TensorSpec(shape=(30, 299, 299, 3), dtype=tf.float32, name='conv2d_29_input_10'), TensorSpec(shape=(30, 1), dtype=tf.float32, name='dense_35_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for conv2d_29_input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 58.31293487548828 secs
                                                   
10/10 [==============================]             
 - 183s 18s/step - loss: 1.1348 - sparse_categorical_accuracy: 0.6664 - f1_score: 0.8419 - val_loss: 0.6958 - val_sparse_categorical_accuracy: 0.6375 - v

Trainable params: 33,752                                                     
Non-trainable params: 658                                                    
_________________________________________________________________            
None                                                                         
 50%|█████     | 1/2 [03:57<03:53, 233.19s/it, best loss: 0.6621878743171692]INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 14199058594203781822)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14049723852272536769)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:

                                                                            
10/10 [==============================]                                       
 - 21s 2s/step - loss: 1.7958 - sparse_categorical_accuracy: 0.5496 - f1_score: 0.8535 - val_loss: 0.6401 - val_sparse_categorical_accuracy: 0.7250 - val_f1_score: 0.8406

100%|██████████| 2/2 [08:02<00:00, 238.13s/it, best loss: 0.6401130557060242]


In [ ]:
best

To Do:
* Write best to disk
* Write individual steps to disk as csv
* Implement a "predict" function

# Ensembling the models

In [ ]:
# parameters:
NUM_MODELS = 10

# build models
# config_file_ensemble = placeholder
models = []

for i in range(NUM_MODELS):
    models[i] = cnn_model(x_train_url, y_train_url, 2, config_v2, name="ensemble_model_v1_{}".format())
    models[i].train(on_tpu=True, epochs= 10, batch_size=256)


# Making Predictions

In [ ]:
LABEL_NAMES = ['t_shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boots']


cpu_model = tpu_model.sync_to_cpu()

from matplotlib import pyplot
%matplotlib inline

def plot_predictions(images, predictions):
  n = images.shape[0]
  nc = int(np.ceil(n / 4))
  f, axes = pyplot.subplots(nc, 4)
  for i in range(nc * 4):
    y = i // 4
    x = i % 4
    axes[x, y].axis('off')
    
    label = LABEL_NAMES[np.argmax(predictions[i])]
    confidence = np.max(predictions[i])
    if i > n:
      continue
    axes[x, y].imshow(images[i])
    axes[x, y].text(0.5, 0.5, label + '\n%.3f' % confidence, fontsize=14)

  pyplot.gcf().set_size_inches(8, 8)  

plot_predictions(np.squeeze(x_test[:16]), 
                 tpu_model.predict(x_test[:16]))

# Misc.

In [ ]:
config_v1 = {
    "name": "vanilla_v1",
    "num_classes": 10,
    "conv_neurons": [32,64,32,16],
    "conv_dropout_layers": [0.]*4,
    "dense_neurons": [100,10],
    "dense_dropout_layers": [0.]*2,
    "kernel_size": (3,3),
    "pool_size": (2,2),
    "activation_fn": "relu",
    "epochs": 1,
    "batch_size": 64,
    "learning_rate": 1e04,
    "optimizer": "adam", #tf.keras.optimizers.Adam,
    "loss": "binary_crossentropy",
}




In [ ]:
# loading Numpy data from GCS
f = BytesIO(file_io.read_file_to_string('gs://data-imr-unisg/np_array_files/x_train.npy', binary_mode=True))
x_train_gs = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://data-imr-unisg/np_array_files/class_labels_trainp.npy', binary_mode=True))
y_train_gs = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://data-imr-unisg/np_array_files/x_test.npy', binary_mode=True))
x_test_gs = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://data-imr-unisg/np_array_files/class_labels_test.npy', binary_mode=True))
y_test_gs = np.load(f)

In [ ]:
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = tf.keras.datasets.mnist.load_data()

x_train_mnist = np.reshape(x_train_mnist, (-1,28,28,1))
x_test_mnist = np.reshape(x_test_mnist, (-1,28,28,1))

In [ ]:
%time  # for mnist training 

config_v2 = {    
    "conv_layers": 1,
    "conv_filters": 16,
    "kernel_size": 3,
    "pool_size":4,
    "dense_layers": 1,
    "dense_neurons": 10,
    "dropout_rate_conv": 0.2,
    "dropout_rate_dense": 0.2,
    "learning_rate": 1e-04,
    "activation_fn": "relu"
}

# x_train_url = 'gs://data-imr-unisg/np_array_files/x_train.npy'
# y_train_url = 'gs://data-imr-unisg/np_array_files/class_labels_trainp.npy'

m2 = cnn_model(x_train_mnist, y_train_mnist, 10,config_v2)
m2.train(on_tpu=True, epochs=40, batch_size=256)

# loading and reshaping MNIST data for our model

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()


In [ ]:
LABEL_NAMES = ['no_car', 'car']


cpu_model = tpu_model.sync_to_cpu()

from matplotlib import pyplot
%matplotlib inline

def plot_predictions(images, predictions):
  n = images.shape[0]
  nc = int(np.ceil(n / 4))
  f, axes = pyplot.subplots(nc, 4)
  for i in range(nc * 4):
    y = i // 4
    x = i % 4
    axes[x, y].axis('off')
    
    label = LABEL_NAMES[np.argmax(predictions[i])]
    confidence = np.max(predictions[i])
    if i > n:
      continue
    axes[x, y].imshow(images[i])
    axes[x, y].text(0.5, 0.5, label + '\n%.3f' % confidence, fontsize=14)

  pyplot.gcf().set_size_inches(8, 8)  

plot_predictions(np.squeeze(x_test[:16]), 
                 tpu_model.predict(x_test[:16]))

## Trying to save files to GCS

In [ ]:
from os import walk

def get_files(mypath):
    """
    list all files in a folder
    
    args: 
        mypath: directory (str)
    returns:
        f: list of all file paths relative to directory specified
    """
    f = []
    for (dirpath, dirnames, filenames) in walk(mypath):
        for file in filenames:
            f.extend([os.path.join(dirpath,file)])
    return f


def copy_folder_files_to_gs(folder, gs_folder):
    """
    Copy model.h5 over to Google Cloud Storage
    """
    for file in get_files(folder):
        #file_path = os.path.join(folder, "/".join(file.split("/")[:]))
        file_name = file.split("/")[-1]

        with file_io.FileIO(file, mode='r') as input_f:
            with file_io.FileIO(gs_folder, mode='w+') as output_f:
                output_f.write(input_f.read())
                print("Saved {} to GCS".format(logger1))

In [ ]:
logger1 = "./logs/1552722826.7797823/events.out.tfevents.1552722838.hsg-iwi-imr2"
logger11="TPU_first_test (15-03-19).ipynb"
bucket = "gs://data-imr-unisg/a"
logger2 = "logs"
with file_io.FileIO(logger1, mode='r') as input_f:
            with file_io.FileIO(bucket, mode='w+') as output_f:
                output_f.write(input_f.read()
                print("Saved {} to GCS".format(logger1))